In [1]:
#imports
import math
import random
import pandas as pd
import numpy as np
import os
import networkx as nx

In [2]:
data = pd.read_excel("Raw_Data/0120.xlsx", engine="openpyxl" ,header=2,index_col=0)

In [3]:
data

,CITY 1,CITY 2,PASSENGERS \nTO CITY 2,PASSENGERS \nFROM CITY 2,FREIGHT \nTO CITY 2,FREIGHT \nFROM CITY 2,MAIL \nTO CITY 2,MAIL \nFROM CITY 2
S.No.,,,,,,,,
1,ADAMPUR,DELHI,2198,2248,0,1.33947,0,0
2,AGARTALA,BENGALURU,4294,4607,0.02,4.251,0,0.862
3,AGARTALA,CHENNAI,2320,1373,0.1,0.771,0,0.482
4,AGARTALA,COIMBATORE,-,5,-,0.004,-,0
5,AGARTALA,DELHI,5467,5823,1.59,19.371,1.002,6.683
...,...,...,...,...,...,...,...,...
538,VIJAYAWADA,VISAKHAPATNAM,2419,2196,0.02015,1.4065,0,0
SUB TOTAL,NaN,NaN,6348701,6398890,29749.5,28587,1306.03,1304.14
TOTAL,NaN,NaN,12747591,NaN,58336.4,NaN,2610.17,NaN


In [4]:
class Graph:
    """
    Implementation of an Weighted, Directed Graph using a matrix
    (Assuming no self loops, and 0 indexing) 
    (Valid only for positive weights)
    """
    def __init__(self, vertices, graph=None):
        self.v = vertices
        if(graph == None):
            self.graph = [[0 for i in range(self.v)] for j in range(self.v)]
        else:
            self.graph = graph

    def add_edge(self, a, b, w):
        # edge directed FROM a TO b
        # w is the weight of the edge
        #method to add an edge
        self.graph[a][b] = w

    def __str__(self):
        return self.graph

    def degree(self, n):
        # returns the indegree and outdegree of node n
        # returns a tuple where first element is indegree, second element is outdegree
        indegree = 0
        outdegree = 0
        for i in range(self.v):
            if(self.graph[n][i] >= 0):
                outdegree += 1
            if(self.graph[i][n] >= 0):
                indegree += 1
        return (indegree, outdegree)



In [5]:
excel_list1 = os.listdir("Raw_Data")
excel_list = []

for i in excel_list1:
    if(i.endswith(".xlsx")):
        excel_list.append(i)
        

In [6]:
for i in excel_list:
    path = "Raw_Data/"+i
    excel = data = pd.read_excel(path, engine="openpyxl" ,header=2,index_col=0, usecols="A:E")
    print(excel)

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


               CITY1          CITY2  PASSENGERS TO CITY 2  \
S.No.                                                       
1           AGARTALA        CHENNAI                  1142   
2      AGATTI ISLAND      BENGALURU                   182   
3      AGATTI ISLAND          KOCHI                   778   
4               AGRA          DELHI                     0   
5               AGRA      KHAJURAHO                   392   
...              ...            ...                   ...   
301         VADODARA      HYDERABAD                  1092   
302         VADODARA        KOLKATA                   743   
303         VADODARA         MUMBAI                 23156   
304       VIJAYAWADA  VISAKHAPATNAM                   400   
TOTAL            NaN            NaN               3790951   

       PASSENGERS FROM CITY 2  
S.No.                          
1                         854  
2                         124  
3                         729  
4                          64  
5             

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


                                                           CITY 1  \
S.No.                                                               
1                                                         ADAMPUR   
2                                                        AGARTALA   
3                                                   AGATTI ISLAND   
4                                                   AGATTI ISLAND   
5                                                            AGRA   
...                                                           ...   
510                                                      VADODARA   
511                                                    VIJAYAWADA   
SUB TOTAL                                                     NaN   
TOTAL                                                         NaN   
Note: City pair wise domestic freight and mail ...            NaN   

                                                           CITY 2  \
S.No.                            

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


...

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


                                                           CITY 1  \
S.No.                                                               
1                                                         ADAMPUR   
2                                                        AGARTALA   
3                                                        AGARTALA   
4                                                        AGARTALA   
5                                                   AGATTI ISLAND   
...                                                           ...   
476                                                      VADODARA   
477                                                    VIJAYAWADA   
SUBTOTAL                                                      NaN   
TOTAL                                                         NaN   
Note: City pair wise domestic freight and mail ...            NaN   

                                                           CITY 2  \
S.No.                            

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


                                                           CITY 1  \
S.No.                                                               
1                                                         ADAMPUR   
2                                                        AGARTALA   
3                                                        AGARTALA   
4                                                   AGATTI ISLAND   
5                                                            AGRA   
...                                                           ...   
480                                                      VADODARA   
481                                                    VIJAYAWADA   
SUBTOTAL                                                      NaN   
TOTAL                                                         NaN   
Note: City pair wise domestic freight and mail ...            NaN   

                                                           CITY 2  \
S.No.                            

PermissionError: [Errno 13] Permission denied: 'Raw_Data/~$0120.xlsx'